# Exploring EM estimates of Fellegi-Sunter parameters from datasets with known parameters

[`splink_data_generation`](https://github.com/moj-analytical-services/splink_data_generation) is a Python package that is able to generate datasets with known parameters.

It has two main ways of generating data:
- `generate_df_gammas_exact`, which produces a dataset with parameters that precisely match the `m` and `u` values specified, and that precisely obeys the assumption of independence of comparison of linking variables conditional on match status.  This function has three limitations:  

    (1) It's not possible to control the overall proportion of matches in the output dataset, and 
    
    (2) It's not possible to generate data that breaks the assumption of conditional independence.
    
    (3) As the complexity of the dataset requested increases, the number of rows generated to satisfy the assumptions can be very high

- `generate_df_gammas_random`, which produces rows at random using the data generating mechanism specified by the supplied parameters.  This function allows the user to specify the proportion of matches, and a covariance matrix which dictates correlations between linking variables conditional on match status.  Since rows are generated at random, the parameters of the resultant dataset will converge to true parameters and the number of rows generated tends to infinity.  



In this notebook, I demonstrate the ability of the EM algorithm to correctly estimate parameters.  I also explore convergence and demonstrate the possibilty that there can sometimes be several local optima, and the EM algorithm doesn't necessarily converge to the parameters of the data generating mechanism.


## Generate the data

We start by using a `splink` settings object (see [editor](https://moj-analytical-services.github.io/splink_settings_editor/)) to specify the columns for our dataframe and the `m` and `u` probabilities.

I have set a few other options to ensure more precise convergnce, and to make sure that the datasets produced by `splink` retain useful information such as the 'ground truth' label of whether the row is a match according to the data generating process. 

In [2]:
settings = {
    "proportion_of_matches": 0.2,
    "link_type": "dedupe_only",
    "comparison_columns": [
        {
            "col_name": "col_1",
            "m_probabilities": [0.3, 0.7],  # Probability of typo
            "u_probabilities": [0.9, 0.1],  # Probability of collision
        },
        {
            "col_name": "col_2",
            "m_probabilities": [0.1, 0.9],  # Probability of typo
            "u_probabilities": [0.975, 0.025],  # Probability of collision
        },
        {
            "col_name": "col_3",
            "m_probabilities": [0.05, 0.95],  # Probability of typo
            "u_probabilities": [0.8, 0.2],  # Probability of collision
        },
    ],
    "max_iterations": 200,
    "em_convergence": 0.0001,
     "additional_columns_to_retain": [
        "true_match", "true_match_probability"
    ]
}

I now use `splink_data_generation` to generate some data according to these `m` and `u` values:

In [3]:
from splink_data_generation.generate_data_exact import generate_df_gammas_exact
from splink_data_generation.match_prob import add_match_prob
from splink_data_generation.log_likelihood import add_log_likelihood
df = generate_df_gammas_exact(settings)
df = add_match_prob(df, settings)
df = add_log_likelihood(df, settings)
cols = [c for c in df.columns if "_r" not in c]
df[cols].head()

/Users/robinlinacre/anaconda3/lib/python3.8/site-packages/splink_data_generation/generate_data_exact.py:130: UserWarning: Note that the proportion_of_matches setting is ignored by this generator. Only the m_probabilities and u_probabilities are observed. The proportion of matches in the generated dataset was 0.500
  warnings.warn(


,gamma_col_1,gamma_col_2,gamma_col_3,true_match_l,unique_id_l,true_match_probability_l,true_log_likelihood_l
0,0,0,0,1,b3bbf501,0.002132,-1.044835
1,0,1,0,1,07874d3d,0.428571,-4.150915
2,0,1,0,1,9d723585,0.428571,-4.150915
3,0,1,0,1,265b8dce,0.428571,-4.150915
4,0,1,0,1,0cd9d9d5,0.428571,-4.150915


In [4]:
from IPython.display import display, Markdown
import numpy as np


binary_prop = df["true_match_l"].mean()
prob_prop = df["true_match_probability_l"].mean()
log_likelihood = sum(df["true_log_likelihood_l"])

md = f"""
The number of rows in the simulated dataset is {len(df):,.0f}

The proportion of matches according to `true_match` status is {binary_prop:,.4f}

The expected proportion of matches according to `true_match_probability` status is {prob_prop:,.4f}

The log likelihood of the dataset given the true parameters is {log_likelihood:,.2f}
"""

display(Markdown(md))


The number of rows in the simulated dataset is 4,000

The proportion of matches according to `true_match` status is 0.5000

The expected proportion of matches according to `true_match_probability` status is 0.5000

The log likelihood of the dataset given the true parameters is -6,507.16


We can see there are correlations amongst the values in comparison vectors:

In [21]:
df[["gamma_col_1", "gamma_col_2", "gamma_col_3"]].corr()

,gamma_col_1,gamma_col_2,gamma_col_3
gamma_col_1,1.000000,0.537339,0.464535
gamma_col_2,0.537339,1.000000,0.665635
gamma_col_3,0.464535,0.665635,1.000000


But as expected there are no correlations when conditioning on match status:

In [25]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
df_m = df[df["true_match_l"]==1]
df_m[["gamma_col_1", "gamma_col_2", "gamma_col_3"]].corr()

,gamma_col_1,gamma_col_2,gamma_col_3
gamma_col_1,1.0000,0.0000,-0.0000
gamma_col_2,0.0000,1.0000,0.0000
gamma_col_3,-0.0000,0.0000,1.0000


## Estimate parameters using Splink.

### Test 1: Use starting parameters equal to true parameters


In this experiment, we give Splink starting parameters equal to the correct parameters.  We expect the algorithm to converge immediately

In [5]:
settings_2 = {
    "proportion_of_matches": 0.5,
    "link_type": "dedupe_only",
    "comparison_columns": [
        {
            "col_name": "col_1",
            "m_probabilities": [0.3, 0.7],  
            "u_probabilities": [0.9, 0.1],  
        },
        {
            "col_name": "col_2",
            "m_probabilities": [0.1, 0.9],  
            "u_probabilities": [0.975, 0.025], 
        },
        {
            "col_name": "col_3",
            "m_probabilities": [0.05, 0.95],  
            "u_probabilities": [0.8, 0.2],  
        },
    ],
     "additional_columns_to_retain": [
        "true_match", "true_match_probability"
    ]
}

In [6]:
import logging 
logging.basicConfig()  # Means logs will print in Jupyter Lab

# Set to DEBUG if you want splink to log the SQL statements it's executing under the hood
logging.getLogger("splink").setLevel(logging.INFO)

from pyspark.context import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [7]:
# Now use Splink to estimate the params from the data
from copy import deepcopy
from splink_data_generation.estimate_splink import estimate

df_e, linker = estimate(df, settings_2 ,spark)
df_e.toPandas().head(5)

/Users/robinlinacre/anaconda3/lib/python3.8/site-packages/splink/case_statements.py:17: UserWarning: The jaro_winkler_sim user definined function is not available in Spark Or you did not pass 'spark' (the SparkSession) into 'Params' Falling back to using levenshtein in the default string comparison functions You can import these functions using the scala-udf-similarity-0.0.6.jar provided with Splink
  warnings.warn(f"The jaro_winkler_sim user definined function is not available in Spark "
INFO:splink.expectation_step:Log likelihood for iteration 0:  -6507.157544158125
INFO:splink.iterate:Iteration 0 complete
INFO:splink.params:The maximum change in parameters was 2.384185793236071e-08 for key π_gamma_col_1_prob_dist_non_match_level_0_probability
INFO:splink.iterate:EM algorithm has converged
INFO:splink.expectation_step:Log likelihood for iteration 1:  -6507.157577685738


,match_probability,unique_id_l,unique_id_r,gamma_col_1,prob_gamma_col_1_non_match,prob_gamma_col_1_match,gamma_col_2,prob_gamma_col_2_non_match,prob_gamma_col_2_match,gamma_col_3,prob_gamma_col_3_non_match,prob_gamma_col_3_match,true_match_l,true_match_r,true_match_probability_l,true_match_probability_r
0,0.002132,b3bbf501,de3c2715,0,0.9,0.3,0,0.975,0.1,0,0.8,0.05,1,1,0.002132,0.002132
1,0.428571,07874d3d,aed91f0f,0,0.9,0.3,1,0.025,0.9,0,0.8,0.05,1,1,0.428571,0.428571
2,0.428571,9d723585,f2ffbdea,0,0.9,0.3,1,0.025,0.9,0,0.8,0.05,1,1,0.428571,0.428571
3,0.428571,265b8dce,9b56092c,0,0.9,0.3,1,0.025,0.9,0,0.8,0.05,1,1,0.428571,0.428571
4,0.428571,0cd9d9d5,30495cc8,0,0.9,0.3,1,0.025,0.9,0,0.8,0.05,1,1,0.428571,0.428571


In [8]:
linker.params

λ (proportion of matches) = 0.5
------------------------------------
gamma_col_1: Comparison of col_1

Probability distribution of gamma values amongst matches:
    value 0: 0.300000 (level represents lowest category of string similarity)
    value 1: 0.700000 (level represents highest category of string similarity)

Probability distribution of gamma values amongst non-matches:
    value 0: 0.900000 (level represents lowest category of string similarity)
    value 1: 0.100000 (level represents highest category of string similarity)
------------------------------------
gamma_col_2: Comparison of col_2

Probability distribution of gamma values amongst matches:
    value 0: 0.100000 (level represents lowest category of string similarity)
    value 1: 0.900000 (level represents highest category of string similarity)

Probability distribution of gamma values amongst non-matches:
    value 0: 0.975000 (level represents lowest category of string similarity)
    value 1: 0.025000 (level repres

As expected, Splink does not iterate away from the correct answer.  

We can see that the EM algorithm does not iterate away from the true parameters, and the log likeilhood of the estimated model is equal to the true log likelihood

What happens if we estimate the model using a different set of starting parameters?

### Test 2: Does Splink correctly estimate parameters with default starting parameters

In [9]:
from copy import deepcopy
settings_3 = {
    "link_type": "dedupe_only",
    "comparison_columns": [
        {
            "col_name": "col_1"
        },
        {
            "col_name": "col_2"
        },
        {
            "col_name": "col_3" 
        },
    ],
    "max_iterations": 200,
    "em_convergence": 0.0001,
     "additional_columns_to_retain": [
        "true_match", "true_match_probability"
    ]
}

settings_copy = deepcopy(settings_3)
df_e, linker = estimate(df, settings_3, spark)
linker.params

/Users/robinlinacre/anaconda3/lib/python3.8/site-packages/splink/case_statements.py:17: UserWarning: The jaro_winkler_sim user definined function is not available in Spark Or you did not pass 'spark' (the SparkSession) into 'Params' Falling back to using levenshtein in the default string comparison functions You can import these functions using the scala-udf-similarity-0.0.6.jar provided with Splink
  warnings.warn(f"The jaro_winkler_sim user definined function is not available in Spark "
INFO:splink.expectation_step:Log likelihood for iteration 0:  -7206.433293015898
INFO:splink.iterate:Iteration 0 complete
INFO:splink.params:The maximum change in parameters was 0.14686706066131594 for key π_gamma_col_3_prob_dist_non_match_level_0_probability
INFO:splink.expectation_step:Log likelihood for iteration 1:  -6552.875324091865
INFO:splink.iterate:Iteration 1 complete
INFO:splink.params:The maximum change in parameters was 0.03220529854297638 for key π_gamma_col_1_prob_dist_match_level_0_pr

λ (proportion of matches) = 0.4996649920940399
------------------------------------
gamma_col_1: Comparison of col_1

Probability distribution of gamma values amongst matches:
    value 0: 0.299742 (level represents lowest category of string similarity)
    value 1: 0.700258 (level represents highest category of string similarity)

Probability distribution of gamma values amongst non-matches:
    value 0: 0.899855 (level represents lowest category of string similarity)
    value 1: 0.100144 (level represents highest category of string similarity)
------------------------------------
gamma_col_2: Comparison of col_2

Probability distribution of gamma values amongst matches:
    value 0: 0.099721 (level represents lowest category of string similarity)
    value 1: 0.900279 (level represents highest category of string similarity)

Probability distribution of gamma values amongst non-matches:
    value 0: 0.974693 (level represents lowest category of string similarity)
    value 1: 0.02530

In [10]:
from splink.expectation_step import _calculate_log_likelihood_df
df_3 = _calculate_log_likelihood_df(df_e, linker.params, spark).toPandas()
df_3["log_likelihood"].sum()


-6507.159182916064

We see that the EM algorithm has converged to the correct answer